## Spotify Different Models for Recommendation 

Let's get firstly libraries

In [1]:
from __future__ import print_function    # (at top of module)
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import time
import sys
import pandas as pd
from urllib.request import Request, urlopen
import json
from bs4 import BeautifulSoup
import re
import urllib.parse
from nltk.corpus import stopwords
from TurkishStemmer import TurkishStemmer
from difflib import SequenceMatcher
from glob import glob
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from math import pi
import sklearn.preprocessing as preprocessing
import numpy as np
import pickle 
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from TurkishStemmer import TurkishStemmer
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 10000)
pd.options.display.max_colwidth = 10000
import subprocess
import sys


In [3]:
sys.path.append("./helper_updater_codes/")
from matrixExporter import matrixExporter

In [4]:

links = []

cid = 'c95dffd5696043ddaa161badb6f9659d'
csec = '7b0b5645d2c44f7ca1f9b9917d23b281'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=csec)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

def getTID(url): # A function to get Spotify ID from given URL
    tid = url.split('/')[-1]
    return 'spotify:track:'+tid
                    

def getSongsDf(artist_name): # A function that get's first 50 (max given by api) songs by the artist. Input: the name of the artist
    results = sp.search(q=artist_name, limit=50) # Query 
    tids = []
    for i, t in enumerate(results['tracks']['items']):
#         print(' ', i, t['name'])
        tids.append(t['uri'])
    start = time.time()
    features = sp.audio_features(tids)
    delta = time.time() - start
    songs  = pd.DataFrame(features)
    songs['name'] = [d['name'] for d in results['tracks']['items']]
    songs['duration_ms'] = [d['duration_ms'] for d in results['tracks']['items']]
    songs['artist_name'] = artist_name
    return songs

In [5]:
fields_to_show = ['name', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'tempo', 'valence']
feature_list_to_show = ['name', 'artist_name', 'duration_ms',
                        'valence', 'acousticness', 'danceability', 'energy',
                        'instrumentalness', 'liveness', 'loudness']
fields = ['acousticness',
         'danceability',
         'energy',
         'instrumentalness',
         'liveness',
         'speechiness',
         'valence']
valence_fields_to_show = ['name','artist_name','valence']
valence_fields_to_show_tr = ['Şarkı', 'Sanatçı', 'Mutluluk Değeri']

analyzeFeatures = ['acousticness',
 'danceability',
 'energy',
 'instrumentalness',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'valence']


Let's create main dataframes 

In [6]:
popularSongFramesFinal = pickle.load(open('.//data//save.p', 'rb'))
allSongsRepeated = pd.concat(popularSongFramesFinal, axis = 'rows')
allSongsUnique = allSongsRepeated.drop_duplicates('Track Name')
allSongsUnique = allSongsUnique.set_index('Track Name')
allSongsUnique.to_csv('.//data//uniqueSongs.csv')#Updates saved unique song


In [6]:
lyricsDf = pd.read_csv('.//otherData//uniqueLyrics.csv')[['lyrics', 'lang', 'Track Name']]
lyricsDf =lyricsDf.set_index('Track Name')
allSongsUnique['lyrics'] = lyricsDf['lyrics']
allSongsUnique['lang'] = lyricsDf['lang']

In [38]:
allSongsUnique.groupby('country_origin').count()

,Artist,Position,Streams,URL,acousticness,analysis_url,danceability,date,duration_ms,energy,...,mode,speechiness,stream_share,tempo,tid,time_signature,track_href,type,uri,valence
country_origin,,,,,,,,,,,,,,,,,,,,,
ca,226,226,226,226,226,226,226,226,226,226,...,226,226,0,226,226,226,226,226,226,226
ch,1056,1056,1056,1056,1056,1056,1056,1056,1056,1056,...,1056,1056,0,1056,1056,1056,1056,1056,1056,1056
gb,1415,1415,1415,1415,1414,1414,1414,1415,1414,1414,...,1414,1414,0,1414,1415,1414,1414,1414,1414,1414
global,1308,1308,1308,1308,1304,1304,1304,1308,1304,1304,...,1304,1304,0,1304,1308,1304,1304,1304,1304,1304
tr,1595,1595,1595,1595,1594,1594,1594,1595,1594,1594,...,1594,1594,1595,1594,1595,1594,1594,1594,1594,1594
us,805,805,805,805,802,802,802,805,802,802,...,802,802,0,802,805,802,802,802,802,802


In [7]:
len(lyricsDf)

1595

In [10]:
len(allSongsUnique[allSongsUnique['country_origin'] == 'tr'])

1595

In [9]:
print('in total there are {0} different songs'.format(len(allSongsUnique)))

in total there are 1595 different songs


In [10]:
print('the labels are: ')
allSongsRepeated.columns

the labels are: 


Index(['Artist', 'Position', 'Streams', 'Track Name', 'URL', 'acousticness',
       'analysis_url', 'danceability', 'date', 'duration_ms', 'energy', 'id',
       'instrumentalness', 'key', 'list_origin', 'liveness', 'loudness',
       'mode', 'speechiness', 'stream_share', 'tempo', 'tid', 'time_signature',
       'track_href', 'type', 'uri', 'valence'],
      dtype='object')

## PART I) Prediction of the nearest popular song of a query song (in terms of musicality)

Here we create a model for recommending nearest song to the query song in terms of 9 features (check analyzeFeatures variable)

Let's create the X matrix

In [30]:
X = allSongsUnique[analyzeFeatures].dropna()


Scale the X matrix

In [31]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

Train

In [33]:
nbrs_musical = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X)

### QUERY

In [34]:
query = 'Son sabah'

In [35]:
results = sp.search(q=query, limit=1)
tids = []

In [36]:
for i, t in enumerate(results['tracks']['items']):
    print('Song is found')
    print('Song Name : ' + t['name'])
    print('Artist Name : ' +t['artists'][0]['name'])
    tids.append(t['uri'])
    
qFeatures = pd.DataFrame(sp.audio_features(tids))[analyzeFeatures]
qFeatures = scaler.transform(qFeatures)

Song is found
Song Name : Son Sabah
Artist Name : mor ve ötesi


In [37]:
distances, indices = nbrs_musical.kneighbors(qFeatures)

In [29]:
for i in indices:
    print(allSongsUnique.iloc[i][['Artist', 'URL']])
    print(distances)

                                    Artist  \
Track Name                                   
Lonely This Christmas                  Mud   
Beşiktaş Tribün Marşı      Various Artists   
Sciroppo (feat. DrefGold)    Sfera Ebbasta   
Machika                           J Balvin   

                                                                             URL  
Track Name                                                                        
Lonely This Christmas      https://open.spotify.com/track/0leIhGny91OrU4mjcshsjI  
Beşiktaş Tribün Marşı      https://open.spotify.com/track/3HSK7PLGOLvYeqnwiypDAk  
Sciroppo (feat. DrefGold)  https://open.spotify.com/track/4VVG3HBGaqSNZqIpmewIA6  
Machika                    https://open.spotify.com/track/6MNGxu8MprKGkchUchkJsy  
[[93.88390959 94.5015408  94.61001462 94.67905857]]


## PART II) Prediction of the nearest popular song of a query string(in terms of lyrics) (Turkish)

Here the input is any string, output is the closest song in terms of lyrics 


Some functions to use 

In [19]:
stemmer = TurkishStemmer()

def cleanText(data): #Cleans text deletes \r \n [] etc. 
    data = str(data)
    data = data.replace(r'\r', ' ')
    data = data.replace(r'\n', ' ')
    p = re.compile(r'<.*?>')
    data = p.sub('', data)
    p = re.compile(r'\[.*?\]')
    data =   p.sub('', data)
    p = re.compile(r'[^\w\s]')
    data =  p.sub('', data).lower()
    data = re.sub(' +',' ',data)
    
    
    return data.replace('\n', ' ')

def cleanAndStem(text, stemmer= stemmer): # Uses turkish stemmer
    return " ".join([stemmer.stem(w) for w in cleanText(text).split()])

First let's load all the turkish songs that we got the lyrics 

In [20]:
allSongsUnique['lang'].unique()


array(['tr', 'en', nan, 'bg', 'es', 'tren', 'ko', 'zh-CN', 'pt', 'espt',
       'fr', 'de', 'ja', 'ar', 'nl', 'uk', 'sv'], dtype=object)

In [21]:
turkishSongs =  allSongsUnique[allSongsUnique['lang'] == 'tr']
print('The number of Turkish songs that we got the lyrics is {0}'.format(len(turkishSongs)))
turkishSongs.head()[['Artist','lyrics']]

The number of Turkish songs that we got the lyrics is 433


,Artist,lyrics
Track Name,,
Gece Gölgenin Rahatına Bak,Çağatay Akman,\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\n[Verse]\r\r\r\r\r\r\r\r\r\r\r\nGece Gölgenin rahatına bak\r\r\r\r\r\r\r\r\r\r\r\nBide dön Kaderimin bahtına yar\r\r\r\r\r\r\r\r\r\r\r\nSeni Düşlerin anlayacak da\r\r\r\r\r\r\r\r\r\r\r\nDön memleketin haline bak\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nAldı dünya çantasını\r\r\r\r\r\r\r\r\r\r\r\nGidiyor bıraktı bize fazlasını\r\r\r\r\r\r\r\r\r\r\r\nDönüp bakmaz arkasına\r\r\r\r\r\r\r\r\r\r\r\nBir de gel tat kalbimin bombasını\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nAdaleti koyduk ortasına\r\r\r\r\r\r\r\r\r\r\r\nDön dedi döndük voltasına\r\r\r\r\r\r\r\r\r\r\r\nFakirin paradan haberi yok ama\r\r\r\r\r\r\r\r\r\r\r\nZenginin meyvesini koy votkasına\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nHadi gönlümü vurdun da\r\r\r\r\r\r\r\r\r\r\r\nSöyle kim kimin umrunda\r\r\r\r\r\r\r\r\r\r\r\nYılan yatıyor koynunda\r\r\r\r\r\r\r\r\r\r\r\nBir öpücük kondur boynundan\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\n[Hook]\r\r\r\r\r\r\r\r\r\r\r\nBiraz dur başucumda\r\r\r\r\r\r\r\r\r\r\r\nKanter kalmışım uçurumda\r\r\r\r\r\r\r\r\r\r\r\nAzrail göz kırpsa\r\r\r\r\r\r\r\r\r\r\r\nEcel bize kucak açsa [x2]\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\n[Bridge]\r\r\r\r\r\r\r\r\r\r\r\nSabah gecenin üstünden\r\r\r\r\r\r\r\r\r\r\r\nÜstüne düştüm üstünden\r\r\r\r\r\r\r\r\r\r\r\nÜstüne üstlük küskünler\r\r\r\r\r\r\r\r\r\r\r\nGeceye küskünler\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nKalem kağıda almışsa\r\r\r\r\r\r\r\r\r\r\r\nGüneş geceyi sarmışsa\r\r\r\r\r\r\r\r\r\r\r\nÖlüm kapıda durmuşsa\r\r\r\r\r\r\r\r\r\r\r\nEcel sonumuz olmuşsa\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\n[Verse]\r\r\r\r\r\r\r\r\r\r\r\nGece gölgenin rahatına bak\r\r\r\r\r\r\r\r\r\r\r\nBir de dön kaderimin bahtına bak yar\r\r\r\r\r\r\r\r\r\r\r\nSeni düşlerin anlayacak da\r\r\r\r\r\r\r\r\r\r\r\nDön memleketin haline bak\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nAldı dünya çantasını\r\r\r\r\r\r\r\r\r\r\r\nGidiyor bıraktı bize fazlasını\r\r\r\r\r\r\r\r\r\r\r\nDönüp bakmaz arkasına\r\r\r\r\r\r\r\r\r\r\r\nBir de gel tat kalbimin bombasını\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nAdaleti koyduk ortasına\r\r\r\r\r\r\r\r\r\r\r\nDön dedi döndük voltasına\r\r\r\r\r\r\r\r\r\r\r\nFakirin paradan haberi yok ama\r\r\r\r\r\r\r\r\r\r\r\nZenginin meyvesini koy votkasına\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nHadi gönlümü vurdun da\r\r\r\r\r\r\r\r\r\r\r\nSöyle kim kimin umrunda\r\r\r\r\r\r\r\r\r\r\r\nYılan yatıyor koynunda\r\r\r\r\r\r\r\r\r\r\r\nBir öpücük kondur boynundan\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\n[Hook]\r\r\r\r\r\r\r\r\r\r\r\nBiraz dur başucumda\r\r\r\r\r\r\r\r\r\r\r\nKanter kalmışım uçurumda\r\r\r\r\r\r\r\r\r\r\r\nAzrail göz kırpsa\r\r\r\r\r\r\r\r\r\r\r\nEcel bize kucak açsa [x2]\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\n
Affet,Müslüm Gürses,"\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nEğer seni kırdıysam\r\r\r\r\r\r\r\r\r\r\r\nDarıl bana\r\r\r\r\r\r\r\r\r\r\r\nAma bir gün beni ararsan\r\r\r\r\r\r\r\r\r\r\r\nBak ruhuna\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nBirden gecem tutarsa\r\r\r\r\r\r\r\r\r\r\r\nGüneşi çevir bana\r\r\r\r\r\r\r\r\r\r\r\nSevgilim bağışla\r\r\r\r\r\r\r\r\r\r\r\nBiraz zor olsa da\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nAffet beni akşamüstü\r\r\r\r\r\r\r\r\r\r\r\nGölgem uzarken\r\r\r\r\r\r\r\r\r\r\r\nÖğleden sonra affet\r\r\r\r\r\r\r\r\r\r\r\nNe zaman istersen\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nAffet beni gece vakti\r\r\r\r\r\r\r\r\r\r\r\nAy doğmuş süzülürken\r\r\r\r\r\r\r\r\r\r\r\nSabaha kalmadan affet\r\r\r\r\r\r\r\r\r\r\r\nTam ayrılık derken\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\nÇünkü sen çölüme yağmur oldun\r\r\r\r\r\r\r\r\r\r\r\nSen, geceme gündüz oldun\r\r\r\r\r\r\r\r\r\r\r\nSen, canıma yoldaş oldun\r\r\r\r\r\r\r\r\r\r\r\nSen, kışıma yorgan oldun\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\n"
Yan Benimle,Sıla,"\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\

Now let's create tf_idf matrix and vector (note it might be logical to save this)

In [22]:
matrixExporter = matrixExporter()

In [23]:
matrixExporter.setSampleSet(turkishSongs)

In [24]:
vectorizer, X_tf = matrixExporter.exportMatrix()

TF-IDF
433


We have 421 songs and 5000 features 

In [25]:
nbrsLyrics = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X_tf)

## Query 

In [26]:
query = """Baktım gökte bir kırmızı bir uçak
Bol çelik bol yıldız bol insan
Bir gece Sevgi Duvarını aştık
Düştüğüm yer öyle açık öyle seçik ki
Başucumda bi sen varsın bi de evren
Saymıyorum ölüp ölüp dirilttiklerimi
Yalnızlığım benim çoğul türkülerim
Ne kadar yalansız yaşarsak o kadar iyi"""

Let's see stemmed version of the query, vectorize the query and find nearest neigbour

In [27]:
query = cleanAndStem(query)
print(query)
queryVec = vectorizer.transform([query]).toarray()

bakt gök bir kır bir uçak bol çelik bol yıldız bol insan bir geç sevg duvar aş düştük yer öyl açık öyl seçik ki başuç bi sen var bi de evren saymıyor ölüp ölüp dirilttik yalnızlık ben çoğul türkü ne kadar yalans yaşar o kadar iyi


In [28]:
distances, indices = nbrsLyrics.kneighbors(queryVec)

In [29]:
for i in indices:
    print(turkishSongs.iloc[i][['Artist', 'URL']])
    print(distances)


                         Artist  \
Track Name                        
Intro               Mabel Matiz   
Uzağa Gidelim             Hidra   
Adını Bilen Yazsın    Murat Boz   
İyinin Yanı              Sehabe   

                                                                      URL  
Track Name                                                                 
Intro               https://open.spotify.com/track/0WTyj7Cn8uW3tCxytamYYt  
Uzağa Gidelim       https://open.spotify.com/track/2PbPTdiM3H47Fe9GNMYSmE  
Adını Bilen Yazsın  https://open.spotify.com/track/3ad2Q4P925rKeYmmknUZ7L  
İyinin Yanı         https://open.spotify.com/track/4TfcxDVRkXtTxB7HH9Zmnx  
[[1.         1.28478633 1.29213321 1.31306496]]


## Part III)  Prediction of the nearest popular song of a given son (in terms of lyrics) (Turkish)

Here the input is a song and the output is songs closest in terms of lyrics (Note run part 2 first)

Some libraries and functions

In [30]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from difflib import SequenceMatcher

In [31]:

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def genius_get_artists(artist_name, n_results = 1):
    global links
    token = 'GS742Alm9Wz1kK6WcSbAMIdSfLjUBmDZqvxSRH38tJQftPiF_gapUynol0DfOBNY'
    baseURL = 'https://api.genius.com/search?q='
    requestURL = baseURL + '%20'+artist_name+'&per_page=' + str(n_results) + '&access_token=' + token
    links.append(requestURL)
    req = Request(
        requestURL,
        headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    data = json.loads(webpage)
    return  data

def getLyric(s_url, artistName):
    req = Request(
        s_url,
        headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage)
    lyric =  soup.findAll("div", {"class": "lyrics"})[0].text
    x = soup.find("a", {'class': "header_with_cover_art-primary_info-primary_artist"})

    if similar(artistName, x.text) >= 0.3:
        return lyric
    return  ""

## Query

In [32]:
nowQuery = 'Tarkan'

In [33]:
results = sp.search(q=nowQuery, limit=1)
tids = []

In [34]:
for i, t in enumerate(results['tracks']['items']):
    print('Song found')
    print('Song name : ' + t['name'])
    print(' Arist name : ' + t['artists'][0]['name'])
    artist = t['artists'][0]['name']
    song = t['name']
    lyrics_query = t['name'] +' ' +  t['artists'][0]['name']
    tids.append(t['uri'])

Song found
Song name : Simarik
 Arist name : Tarkan


let's download the lyric for the given song 

In [35]:
data = genius_get_artists(urllib.parse.quote(lyrics_query))
lyricsUrl = data['response']['hits'][0]['result']['url']
lyric = getLyric(lyricsUrl, artistName=artist)

Let's check out the lyrics found for the query song 

In [36]:
lyric

'\n\nTakmış koluna elin adamını\nBeni orta yerimden çatlatıyor\nAğzında sakızı şişirip şişirip\nArsız arsız patlatıyor\nBelki de bu yüzden vuruldum\nSahibin olamadım ya\nSığar mı erkekliğe seni şımarık\nDeğişti mi bu dünya\nÇekmiş kaşına gözüne sürme\nDudaklar kıpkırmızı kırıtıyor\nBir de karşıma geçmiş utanması yok\nİnadıma inadıma sırıtıyor\nBiz böyle mi gördük babamızdan\nEle güne rezil olduk\nYeni adet gelmiş eski köye vah\nDostlar mahvolduk\nSeni gidi fındıkkıran\nYılanı deliğinden çıkaran\nKaderim, püsküllü belam\nYakalarsam\nSeni gidi fındıkkıran\nYılanı deliğinden çıkaran\nKaderim, püsküllü belam\nYakalarsam\nOcağına düştüm yavru\nKucağına düştüm yavru\nSıcağına düştüm yavru\nEl aman\nOcağına düştüm yavru\nKucağına düştüm yavru\nSıcağına düştüm yavru\nEl aman\nTakmış koluna elin adamını\nBeni orta yerimden çatlatıyor\nAğzında sakızı şişirip şişirip\nArsız arsız patlatıyor\nBiz böyle mi gördük babamızdan\nEle güne rezil olduk\nYeni adet gelmiş eski köye vah\nDostlar mahvolduk\nS

Let's see the stemmed version 

In [37]:
query = cleanAndStem(lyric)
print(query)
queryVec = vectorizer.transform([query]).toarray()

tak kol el adam ben or yer çatlatıyor ağz sakız şişirip şişirip arsız arsız patlatıyor belk de bu yüz vurul sahip olam ya sığar mı erkeklik sen şımarık değiş mi bu dünya çek kaş göz sürm dudak kıpkır kırıtıyor bir de karş geç utanma yok i̇nadıma inat sırıtıyor biz böyl mi gör baba el gün rezil ol yen adet gel esk kö vah dost mahvol sen git fındıkkıran yılan delik çıkaran kader püskül belam yaka sen git fındıkkıran yılan delik çıkaran kader püskül belam yaka ocak düşt yavru kucak düşt yavru sıcak düşt yavru el aman ocak düşt yavru kucak düşt yavru sıcak düşt yavru el aman tak kol el adam ben or yer çatlatıyor ağz sakız şişirip şişirip arsız arsız patlatıyor biz böyl mi gör baba el gün rezil ol yen adet gel esk kö vah dost mahvol sen git fındıkkıran yılan delik çıkaran kader püskül belam yaka sen git fındıkkıran yılan delik çıkaran kader püskül belam yaka ocak düşt yavru kucak düşt yavru sıcak düşt yavru el aman ocak düşt yavru kucak düşt yavru sıcak düşt yavru el aman sen git fındıkkıra

In [38]:
distances, indices = nbrsLyrics.kneighbors(queryVec)

In [39]:
for i in indices:
    print(turkishSongs.iloc[i][['Artist', 'URL']])
    print(distances)

                                        Artist  \
Track Name                                       
Intro                              Mabel Matiz   
Gece Gölgenin Rahatına Bak       Çağatay Akman   
Bana Öyle Bakma                         Teoman   
Vah ki Ne Vah               Nil Karaibrahimgil   

                                                                              URL  
Track Name                                                                         
Intro                       https://open.spotify.com/track/0WTyj7Cn8uW3tCxytamYYt  
Gece Gölgenin Rahatına Bak  https://open.spotify.com/track/3P31rcl0ym5paqRdwSiZps  
Bana Öyle Bakma             https://open.spotify.com/track/4DDMXN0aa2RLeR51FX186y  
Vah ki Ne Vah               https://open.spotify.com/track/7cX5CcX6dLYDwqtNPEWJCs  
[[1.         1.34143403 1.34797044 1.34801218]]


## PART IV) Feeling Analysis by lyrics

Let's import feeling dictionary (english)

In [40]:
feelingDf = pd.read_csv('.\\otherData\\NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', header = None, sep = '\t', index_col= 0, names= ['feeling','assoc'])

Let's see the type of doc

In [41]:
feelingDf['word'] = feelingDf.index

Let's import the dictionary for translating to Turkish

In [42]:
dictionary = pd.read_csv('.\\otherData\\dictionary_NRC.csv',header=None)
dictionary = pd.Series(dictionary[1].values,index=dictionary[0]).to_dict()
feelingDf['tr'] = feelingDf['word'].apply(lambda x: dictionary[x])

In [43]:
feelingDf.head()

,feeling,assoc,word,tr
aback,anger,0,aback,pupada
aback,anticipation,0,aback,pupada
aback,disgust,0,aback,pupada
aback,fear,0,aback,pupada
aback,joy,0,aback,pupada


Let's create X matrix

In [44]:
X_feeling = pd.read_csv('feelingX.csv', index_col='Track Name')

In [45]:
X_feeling.head()


,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
Track Name,,,,,,,,,,
1000 Parça,29,18,23,24,14,51,40,29,9,21
#Biziz - feat. Lil Bege,15,12,8,26,14,41,29,26,6,18
2013,3,2,0,7,4,18,7,14,1,0


In [46]:
scaler = StandardScaler()
X_feeling = scaler.fit_transform(X_feeling)

In [47]:
nbrsFeeling = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X_feeling)

## Query

In [48]:
nowQuery = 'tarkan kuzu kuzu'

In [49]:
results = sp.search(q=nowQuery, limit=1)
tids = []
for i, t in enumerate(results['tracks']['items']):
    print('Song found')
    print('Song name : ' + t['name'])
    print(' Arist name : ' + t['artists'][0]['name'])
    artist = t['artists'][0]['name']
    song = t['name']
    lyrics_query = t['name'] +' ' +  t['artists'][0]['name']
    tids.append(t['uri'])

Song found
Song name : Kuzu Kuzu
 Arist name : Tarkan


In [50]:
data = genius_get_artists(urllib.parse.quote(lyrics_query))
lyricsUrl = data['response']['hits'][0]['result']['url']
lyric = getLyric(lyricsUrl, artistName=artist)
lyric = cleanText(lyric)

In [51]:
%%time
out = []

for word in lyric.split()[:]:
    wordOrigin = word
#     print(word)
#     print("-------")
    word = word.lower()
    word = stemmer.stem(word)
#     print(word)
    closest =  feelingDf[feelingDf['tr'].str.startswith(word[0])]['tr'].apply(lambda x: similar(x, word))
    i = closest.argmax()
    if closest.values.max()>0.8:
        out.append(i)
#         print(wordOrigin)
#         print(word)


KeyboardInterrupt: 

In [52]:
queryVec = feelingDf.loc[out].groupby('feeling').sum().transpose().loc[['assoc']]  
queryVec = scaler.transform(queryVec)

In [53]:
distances, indices = nbrsFeeling.kneighbors(queryVec)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 4

In [54]:
for i in indices:
    print(turkishSongs.iloc[i][['Artist', 'URL']])
    print(distances)

                                        Artist  \
Track Name                                       
Intro                              Mabel Matiz   
Gece Gölgenin Rahatına Bak       Çağatay Akman   
Bana Öyle Bakma                         Teoman   
Vah ki Ne Vah               Nil Karaibrahimgil   

                                                                              URL  
Track Name                                                                         
Intro                       https://open.spotify.com/track/0WTyj7Cn8uW3tCxytamYYt  
Gece Gölgenin Rahatına Bak  https://open.spotify.com/track/3P31rcl0ym5paqRdwSiZps  
Bana Öyle Bakma             https://open.spotify.com/track/4DDMXN0aa2RLeR51FX186y  
Vah ki Ne Vah               https://open.spotify.com/track/7cX5CcX6dLYDwqtNPEWJCs  
[[1.         1.34143403 1.34797044 1.34801218]]


## Part V) Most N songs from the artist 

In [55]:
N = 3
artist = 'Barış Manço'
artistsDfDict = dict()
df = getSongsDf(artist).sort_values('valence')[['artist_name', 'name',  'valence']]
realArtistName = df['artist_name'].mode().iloc[0]
df = df[df['artist_name'] == realArtistName]

Saddest N song

In [56]:
df.iloc[:N,:]

,artist_name,name,valence
3,Barış Manço,Dağlar Dağlar,0.109
39,Barış Manço,Uzun İnce Bir Yoldayım,0.120
2,Barış Manço,Gül Pembe,0.256


Happiest N song

In [57]:
df.iloc[-N:,:]

,artist_name,name,valence
25,Barış Manço,Aman Yavaş Aheste,0.941
29,Barış Manço,Hal Hal,0.975
7,Barış Manço,Hal Hal,0.975


## Part VI) Recommend me a random song for my feeling level

In [58]:
duyguLevel = 0
duyguLevel = duyguLevel / 10


In [59]:
allSongsUnique[(allSongsUnique['valence'] < duyguLevel + 0.05) & (allSongsUnique['valence'] > duyguLevel - 0.05)][['Artist', 'valence', 'URL']].sample(5)

,Artist,valence,URL
Track Name,,,
Deli Ozan,Allame,0.0390,https://open.spotify.com/track/633Oe1mFffNTIMlfojXzAd
Migration,Bonobo,0.0392,https://open.spotify.com/track/61g7oQKm8DFEL7r2g9LS8x
Eyes Closed,Halsey,0.0453,https://open.spotify.com/track/1xa5pXOwLoxHuBDU0JiUdn
"""All That Is or Ever Was or Ever Will Be""",Alan Silvestri,0.0338,https://open.spotify.com/track/1Ig5PYAhZaCiCVAdIEcdAC
Gözleri Aşka Gülen,Nilipek.,0.0398,https://open.spotify.com/track/4qzZm5EIdFurBpDieEmVc9
